In [8]:
from pymongo import MongoClient
from pymongo.cursor import Cursor
from datetime import datetime, timedelta
from unidecode import unidecode
from dotenv import load_dotenv

import pandas as pd

import numpy as np
import os
import re
import sys

# Acessando o banco MongoDB Compass
cliente = MongoClient(
    "ddns.cartor.com.br",
    port=57017,
    username='cartor',
    password='Cartor1212',
    authSource='admin',
    authMechanism='DEFAULT'
)

db = cliente.CVM
collection = 'Empresas_fin6'

In [9]:
load_dotenv()   
stocks_path = os.getenv('AMBIENTE_MAC')+'stocks_data.parquet'
df_stocks = pd.read_parquet(stocks_path); df_stocks.fillna(0, inplace=True)

def get_adjclosing_price(df, date, ticker):
    if isinstance(date, str):
        date = pd.to_datetime(date)
    filtered_df = df[(df['data'] == date) & (df['ticker'] == ticker)]
    if not filtered_df.empty:
        return filtered_df['preco_fechamento_ajustado'].iloc[0]
    else:
        return None
    
def encontrar_close(df, stock_name, fdata_divulgacao):
    try:
        filtro = (df['Stock'] == stock_name) & (df['Date'] == fdata_divulgacao)
        close_value = df.loc[filtro, 'Close'].values[0]
        return close_value
    except IndexError:
        try:
            filtro = (df['Stock'] == stock_name) & (df['Date'] == (fdata_divulgacao+timedelta(days=1)))
            close_value = df.loc[filtro, 'Close'].values[0]
            return close_value            
        except IndexError:        
            try:
                filtro = (df['Stock'] == stock_name) & (df['Date'] == (fdata_divulgacao+timedelta(days=2)))
                close_value = df.loc[filtro, 'Close'].values[0]
                return close_value
            except IndexError:
                try:
                    filtro = (df['Stock'] == stock_name) & (df['Date'] == (fdata_divulgacao+timedelta(days=2)))
                    close_value = df.loc[filtro, 'Close'].values[0]
                    return close_value
                except IndexError:                                                                
                    return None 
    
def converterstr_data(fData_hora_string):
    data_hora = datetime.strptime(fData_hora_string, "%d/%m/%Y %H:%M:%S")
    data = data_hora.date()
    return data    

def extrair_hora(fData_hora_str):
    data_hora = datetime.strptime(fData_hora_str, "%d/%m/%Y %H:%M:%S")
    horas = data_hora.hour
    if horas > 17:
        aumentar_dias = 1
    else:
        aumentar_dias = 0
    return aumentar_dias


In [4]:
def finddoc_byequity(fDb, fCollection_name, fEquity): # acha o documento da empresa pelo ticker da bolsa 
    count = 0
    while count != 1:
        collection = fDb[fCollection_name]
        documents = collection.find({'equity': fEquity})
        count = collection.count_documents({'equity': fEquity})
        if count > 1:
            print('Lista encontrada: ')
            print(type(documents))
            for doc in documents:
                print(doc['_id'])
            print('CODIGO EXISTENTE EM MAIS DE 01 DOCUMENTO')
            exit()
        if count == 0:
            print("Não foi encontrada a empresa desejada.")
            return None  # Retorna None se não encontrar a empresa
    return documents

def finddoc_byname(fDb, fCollection_name, fContaining_word, fTrimestre=False):
    count = 0
    while count != 1:
        if fTrimestre:
            collection = fDb[fCollection_name]
            regex = re.compile(fContaining_word+fTrimestre, re.IGNORECASE)
            documents = collection.find({'_id': regex})
            count = collection.count_documents({'_id': regex})
        else:
            collection = fDb[fCollection_name]
            regex = re.compile(fContaining_word, re.IGNORECASE)
            documents = collection.find({'_id': regex})
            count = collection.count_documents({'_id': regex})
        if count > 1:
            print(type(documents))
            print('erro aconteceu - checar base')
            for doc in documents:
                print(doc['_id'])
            break
        if count == 0:
            print("Não foi encontrada a empresa desejada.")
            return None
    return documents

def finddoc_by_codcvm(fDb, fCollection_name, fCodcvm, fContaining_word, fTrimestre): 
    count = 0
    while count != 1:
        collection = fDb[fCollection_name]
        documents = collection.find({'cod_cvm': fCodcvm})
        count = collection.count_documents({'cod_cvm': fCodcvm})
        if count > 1:
            regex = re.compile(fContaining_word+fTrimestre, re.IGNORECASE)
            print(regex)
            for doc in documents:
                if fTrimestre in doc['_id']:
                    documents = doc
                    count = 1
                    break
            print('CODIGO EXISTENTE EM MAIS DE 01 DOCUMENTO')
        if count == 0:
            print("Não foi encontrada a empresa desejada.")
            return None      
    if isinstance(documents, Cursor):
        return documents.next()
    else: 
        return documents

def upload_infofin(fData_fin): # retorna os trimestres da empresa 
    trimestres_upload = []
    for chave, valor in fData_fin.items():
        if isinstance(chave, str) and chave.startswith('2'):
            trimestres_upload.append(chave)
    trimestres_upload = sorted(trimestres_upload, reverse=True)     
    tipo_balanco, trimestres_upload = get_balance_type(fData_fin, trimestres_upload)
    if tipo_balanco == 'ERROR': 
        print('Erro na captura do balanço'); sys.exit()
    tipo_balanco2 = re.sub(r'(DF)(Consolidado|Individual)', r'\1 \2', tipo_balanco)
    return tipo_balanco, tipo_balanco2, trimestres_upload

def balance_type(fEmpresa, fTrimestre): # seleciona o tipo de balanco existente
    tipo_balanco = 'ERROR'   
    for key in fEmpresa[fTrimestre[0]]:
        key2 = re.sub(r'(DF)(Consolidado|Individual)', r'\1 \2', key)       
        try:
             pvalor = fEmpresa[fTrimestre[-1]][key][key2+' - Demonstração do Resultado']['3.01']['vl_conta']
             if pvalor != 0: 
                 tipo_balanco = key
        except KeyError:
             pvalor = fEmpresa[fTrimestre[0]][key][key2+' - Demonstração do Resultado']['3.01']['vl_conta']
             if pvalor != 0: 
                 tipo_balanco = key
                 break
    novalista_trimestres = []
    if tipo_balanco == 'DFConsolidado':
        for trimestre in fTrimestre:
            if len(fEmpresa[trimestre].get(tipo_balanco, {})) != 0:
                    novalista_trimestres.append(trimestre) 
        fTrimestre = novalista_trimestres
    return tipo_balanco, fTrimestre

def get_balance_type(fEmpresa, fTrimestre): # seleciona o tipo de balanco existente
    tipo_balanco = 'ERROR'
    for key in fEmpresa[fTrimestre[0]]:
        key2 = re.sub(r'(DF)(Consolidado|Individual)', r'\1 \2', key)       
        # print(key)
        # print(key2)        
        try:
            nr_trimestre = 0
            while nr_trimestre < 8:
                pvalor = fEmpresa[fTrimestre[nr_trimestre]][key][key2+' - Demonstração do Resultado']['3.01']['vl_conta']
                if pvalor !=0: 
                    nr_trimestre+=1
                else:
                    break
            if nr_trimestre == 8:
                tipo_balanco = key
            # print(key)
            # print(nr_trimestre)

        except KeyError:
            print(f'Empresa não possui {key} suficiente')
        #      pvalor = fEmpresa[fTrimestre[0]][key][key2+' - Demonstração do Resultado']['3.01']['vl_conta']
        #      if pvalor != 0: 
        #          tipo_balanco = key
        #          break
        if tipo_balanco != 'ERROR': break
    
    novalista_trimestres = []
    # if tipo_balanco == 'DFConsolidado':
    # for trimestre in fTrimestre:
    for i, trimestre in enumerate(fTrimestre):        
        if len(fEmpresa[trimestre].get(tipo_balanco, {})) != 0:
                novalista_trimestres.append(trimestre) 
    fTrimestre = novalista_trimestres
    return tipo_balanco, fTrimestre

def encontrar_conta(fMydict, fConta):
   f_value = 0 
   for key in fMydict:
      string = unidecode(fMydict[key]['ds_conta'].lower())
      if fConta in string:
         f_value = fMydict[key]['vl_conta']
         break
   return f_value

def transform_Dfp_tri(fDicionario, fConta, fQuarter, fQtdtrimestres, fTrimestres, fTipobalanco, fTipobalanco2):
    f_valor = 0    
    total_tri = fQuarter+3
    f_valor = fDicionario[fTrimestres[fQuarter]][fTipobalanco][fTipobalanco2+' - Demonstração do Resultado'][fConta]['vl_conta']
    dt = datetime.strptime(fTrimestres[fQuarter], '%Y-%m-%d')
    if dt.month == 12:  
        if total_tri > fQtdtrimestres:
            f_valor /= 4
        else:
            f_tri_1 = fDicionario[fTrimestres[fQuarter+1]][fTipobalanco][fTipobalanco2+' - Demonstração do Resultado'][fConta]['vl_conta']
            f_tri_2 = fDicionario[fTrimestres[fQuarter+2]][fTipobalanco][fTipobalanco2+' - Demonstração do Resultado'][fConta]['vl_conta']
            f_tri_3 = fDicionario[fTrimestres[fQuarter+3]][fTipobalanco][fTipobalanco2+' - Demonstração do Resultado'][fConta]['vl_conta']
            f_valor -= (f_tri_1+f_tri_2+f_tri_3) 
    return f_valor

def depreciacao_FDC_tri2(fDicionario, fQuarter, fQtdtrimestres, fTrimestres, fTipobalanco, fTipobalanco2): # deletar após testes
    f_valor = 0    
    f_total_tri = fQuarter+1
    dt = datetime.strptime(fTrimestres[fQuarter], '%Y-%m-%d')
    if f_total_tri+1 >= fQtdtrimestres:
        try:
            f_valor = abs(encontrar_conta(fDicionario[fTrimestres[fQuarter]][fTipobalanco][fTipobalanco2+' - Demonstração do Fluxo de Caixa (Método Indireto)'], 'depreciacoes'))
            if f_valor==0:
                try:
                    f_valor = abs(encontrar_conta(fDicionario[fTrimestres[fQuarter]][fTipobalanco][fTipobalanco2+' - Demonstração do Fluxo de Caixa (Método Indireto)'], 'depreciacao'))
                except KeyError:
                    f_valor = 0   
        except KeyError:
            f_valor = 0
        dt = datetime.strptime(fTrimestres[fQuarter], '%Y-%m-%d')
        if dt.month == 12: f_valor /= 4
        elif dt.month == 9: f_valor /= 3
        elif dt.month == 6: f_valor /= 2
    else:
        if dt.month==3:
            try:
                f_valor = abs(encontrar_conta(fDicionario[fTrimestres[fQuarter]][fTipobalanco][fTipobalanco2+' - Demonstração do Fluxo de Caixa (Método Indireto)'], 'depreciacoes'))
                if f_valor==0:
                    try:
                        f_valor = abs(encontrar_conta(fDicionario[fTrimestres[fQuarter]][fTipobalanco][fTipobalanco2+' - Demonstração do Fluxo de Caixa (Método Indireto)'], 'depreciacao'))
                    except KeyError:
                        f_valor = 0   
            except KeyError:
                f_valor = 0
        else: 
            # trimestre atual
            try:
                f_valor = abs(encontrar_conta(fDicionario[fTrimestres[fQuarter]][fTipobalanco][fTipobalanco2+' - Demonstração do Fluxo de Caixa (Método Indireto)'], 'depreciacoes'))
                if f_valor==0:
                    try:
                        f_valor = abs(encontrar_conta(fDicionario[fTrimestres[fQuarter]][fTipobalanco][fTipobalanco2+' - Demonstração do Fluxo de Caixa (Método Indireto)'], 'depreciacao'))
                    except KeyError:
                        f_valor = 0   
            except KeyError:
                f_valor = 0
            # trimestre anterior
            f_valor_ant = 0
            try:
                f_valor_ant = abs(encontrar_conta(fDicionario[fTrimestres[fQuarter+1]][fTipobalanco][fTipobalanco2+' - Demonstração do Fluxo de Caixa (Método Indireto)'], 'depreciacoes'))
                if f_valor_ant==0:
                    try:
                        f_valor_ant = abs(encontrar_conta(fDicionario[fTrimestres[fQuarter+1]][fTipobalanco][fTipobalanco2+' - Demonstração do Fluxo de Caixa (Método Indireto)'], 'depreciacao'))
                    except KeyError:
                        f_valor_ant = 0   
            except KeyError:
                f_valor_ant = 0
            f_valor = f_valor - f_valor_ant
    return f_valor

def obter_valor_depreciacao(fDicionario, fTrimestre, fTipobalanco, fTipobalanco2):
    chave = fTipobalanco2 + ' - Demonstração do Fluxo de Caixa (Método Indireto)'
    try:
        valor = abs(encontrar_conta(fDicionario[fTrimestre][fTipobalanco][chave], 'depreciacoes'))
        if valor == 0:
            valor = abs(encontrar_conta(fDicionario[fTrimestre][fTipobalanco][chave], 'depreciacao'))
    except KeyError:
        valor = 0
    return valor

def depreciacao_FDC_tri(fDicionario, fQuarter, fQtdtrimestres, fTrimestres, fTipobalanco, fTipobalanco2):
    f_valor = 0    
    f_total_tri = fQuarter + 1
    dt = datetime.strptime(fTrimestres[fQuarter], '%Y-%m-%d')

    if  dt.month == 3:
        f_valor = obter_valor_depreciacao(fDicionario, fTrimestres[fQuarter], fTipobalanco, fTipobalanco2)
    elif f_total_tri + 1 >= fQtdtrimestres:
        divisor = {'12': 4, '9': 3, '6': 2}.get(str(dt.month), 1)
        f_valor /= divisor
    else:
        f_valor_atual = obter_valor_depreciacao(fDicionario, fTrimestres[fQuarter], fTipobalanco, fTipobalanco2)
        f_valor_anterior = obter_valor_depreciacao(fDicionario, fTrimestres[fQuarter + 1], fTipobalanco, fTipobalanco2) # tratar
        f_valor = f_valor_atual - f_valor_anterior       
    return f_valor

def transform_FDC_tri(fDicionario, fConta, fQuarter, fQtdtrimestres, fTrimestres, fTipobalanco, fTipobalanco2):
    f_total_tri = fQuarter + 1
    dt = datetime.strptime(fTrimestres[fQuarter], '%Y-%m-%d')
    try:
        f_valor = fDicionario[fTrimestres[fQuarter]][fTipobalanco][fTipobalanco2 + ' - Demonstração do Fluxo de Caixa (Método Indireto)'][fConta]['vl_conta']
        if f_total_tri > fQtdtrimestres:
            f_valor /= 4 if dt.month == 12 else 3 if dt.month == 9 else 2 if dt.month == 6 else 1
        elif dt.month != 3 and f_total_tri < len(fTrimestres):
            print(dt.month)
            print(fTrimestres[f_total_tri])
            print(fTrimestres[f_total_tri])            
            f_valor -= fDicionario.get(fTrimestres[f_total_tri], {}).get(fTipobalanco, {}).get(fTipobalanco2 + ' - Demonstração do Fluxo de Caixa (Método Indireto)', {}).get(fConta, {}).get('vl_conta', 0)
            print(f_valor)
        else:
            f_valor = 0
    except KeyError:
        try:
            f_valor = fDicionario[fTrimestres[fQuarter]][fTipobalanco][fTipobalanco2 + ' - Demonstração do Fluxo de Caixa (Método Direto)'][fConta]['vl_conta']
        except KeyError:
            f_valor = 0

    return f_valor

def transform_FDC_tri(fDicionario, fConta, fQuarter, fQtdtrimestres, fTrimestres, fTipobalanco, fTipobalanco2):
    f_total_tri = fQuarter + 1
    dt = datetime.strptime(fTrimestres[fQuarter], '%Y-%m-%d')
    f_valor = fDicionario[fTrimestres[fQuarter]][fTipobalanco][fTipobalanco2 + ' - Demonstração do Fluxo de Caixa (Método Indireto)'][fConta]['vl_conta']
    if dt.month == 3:
        return f_valor
    elif f_total_tri < len(fTrimestres):
        f_valor -= fDicionario.get(fTrimestres[f_total_tri], {}).get(fTipobalanco, {}).get(fTipobalanco2 + ' - Demonstração do Fluxo de Caixa (Método Indireto)', {}).get(fConta, {}).get('vl_conta', 0)
    else:
        if dt.month == 12:
            f_valor /= 4 
        elif dt.month == 9:
            f_valor /= 3 
        else:
            f_valor /= 2
    if f_valor == 0: f_valor = 1
    return f_valor

In [10]:
info_input = []
stock = 'MATD3'; info_input.append(stock)

In [11]:
def get_balance_type(fEmpresa, fTrimestre): # seleciona o tipo de balanco existente
    tipo_balanco = 'ERROR'
    for key in fEmpresa[fTrimestre[0]]:
        key2 = re.sub(r'(DF)(Consolidado|Individual)', r'\1 \2', key)
        try:
            nr_trimestre = 0
            while nr_trimestre < 8:
                pvalor = fEmpresa[fTrimestre[nr_trimestre]][key][key2+' - Demonstração do Resultado']['3.05']['vl_conta']
                if pvalor !=0: 
                    nr_trimestre+=1
                else:
                    break
            if nr_trimestre == 8:
                tipo_balanco = key
            # print(key)
            # print(nr_trimestre)

        except KeyError:
            print(f'Empresa não possui {key} suficiente')
        #      pvalor = fEmpresa[fTrimestre[0]][key][key2+' - Demonstração do Resultado']['3.01']['vl_conta']
        #      if pvalor != 0: 
        #          tipo_balanco = key
        #          break
        if tipo_balanco != 'ERROR': break
    
    novalista_trimestres = []
    # if tipo_balanco == 'DFConsolidado':
    # for trimestre in fTrimestre:
    for i, trimestre in enumerate(fTrimestre):        
        if len(fEmpresa[trimestre].get(tipo_balanco, {})) != 0:
                novalista_trimestres.append(trimestre) 
    fTrimestre = novalista_trimestres
    return tipo_balanco, fTrimestre

def upload_infofin(fData_fin): # retorna os trimestres da empresa 
    trimestres_upload = []
    for chave, valor in fData_fin.items():
        if isinstance(chave, str) and chave.startswith('2'):
            trimestres_upload.append(chave) 
    trimestres_upload = sorted(trimestres_upload, reverse=True)     
    tipo_balanco, trimestres_upload = get_balance_type(fData_fin, trimestres_upload)
    if tipo_balanco == 'ERROR': 
        print('Erro na captura do balanço'); sys.exit()
    tipo_balanco2 = re.sub(r'(DF)(Consolidado|Individual)', r'\1 \2', tipo_balanco)
    return tipo_balanco, tipo_balanco2, trimestres_upload

document = finddoc_byequity(db, collection, stock)
doc_stock = document.next() # dicionario referente a stock pick
id_empresa = doc_stock['_id']; print(id_empresa)
tipo_balanco1, tipo_balanco2, trimestres_lista = upload_infofin(doc_stock) 

print(tipo_balanco1)
print(tipo_balanco2)
print(trimestres_lista[-4:])
print(len(trimestres_lista))

HOSPITAL MATER DEI S.A.
DFConsolidado
DF Consolidado
['2022-06-30', '2022-03-31', '2021-12-31', '2021-09-30']
9


In [6]:
ebit = []
for i in range(len(trimestres_lista)):
    ebt = transform_Dfp_tri(doc_stock, '3.05', i, len(trimestres_lista), trimestres_lista, tipo_balanco1, tipo_balanco2)
    financial_result = transform_Dfp_tri(doc_stock, '3.06', i, len(trimestres_lista), trimestres_lista, tipo_balanco1, tipo_balanco2)
    ebit.append(ebt - financial_result)
ebit = np.array(ebit)

print(stock)

# calcular 3t, 2t, 1t variacao
info_input.append(round(ebit[0]/ebit[0+4], 2))
info_input.append(round(ebit[1]/ebit[1+4], 2))
info_input.append(round(ebit[2]/ebit[2+4], 2)) 
info_input.append(round(info_input[2] - info_input[1], 2))
info_input.append(round(info_input[3] - info_input[2], 2))
print(f'Trimestre {trimestres_lista[0]} com variação de {info_input[3]}')
print(f'Trimestre {trimestres_lista[1]} com variação de {info_input[2]}')
print(f'Trimestre {trimestres_lista[2]} com variação de {info_input[1]}')
print(f'Variacao do 2 para o 1 = {info_input[4]}')
print(f'Variacao do 3 para o 2 = {info_input[5]}')

# calcular o ebit ltm
# variacao_3t = round(np.sum(ebit[0:4])/np.sum(ebit[4:4+4]), 2)
# variacao_2t = round(np.sum(ebit[1:1+4])/np.sum(ebit[5:5+4]), 2)
# variacao_1t = round(np.sum(ebit[2:2+4])/np.sum(ebit[6:6+4]), 2)

info_input.append(round(np.sum(ebit[0:4])/np.sum(ebit[4:4+4]), 2)) # 6
info_input.append(round(np.sum(ebit[1:1+4])/np.sum(ebit[5:5+4]), 2)) # 7
info_input.append(round(np.sum(ebit[2:2+4])/np.sum(ebit[6:6+4]), 2)) # 8

# variacao_21ltm = round(variacao_2t - variacao_1t, 2)
# variacao_32ltm = round(variacao_3t - variacao_2t, 2)

info_input.append(round(info_input[7] - info_input[8], 2)) # 9 
info_input.append(round(info_input[6] - info_input[7], 2)) # 10

print(f'Trimestre {trimestres_lista[0]} com variação LTM de {info_input[6]}')
print(f'Trimestre {trimestres_lista[1]} com variação LTM de {info_input[7]}')
print(f'Trimestre {trimestres_lista[2]} com variação LTM de {info_input[8]}')
print(f'Variacao do 2 para o 1 ltm = {info_input[9]}')
print(f'Variacao do 3 para o 2 ltm = {info_input[10]}')

print(f'Total de Ebits {len(ebit)}')
print(f'Total de Trimestres {len(trimestres_lista)}')


VAMO3
Trimestre 2023-09-30 com variação de 2.13
Trimestre 2023-06-30 com variação de 1.67
Trimestre 2023-03-31 com variação de 1.34
Variacao do 2 para o 1 = 0.33
Variacao do 3 para o 2 = 0.46
Trimestre 2023-09-30 com variação LTM de 1.76
Trimestre 2023-06-30 com variação LTM de 2.13
Trimestre 2023-03-31 com variação LTM de 2.45
Variacao do 2 para o 1 ltm = -0.32
Variacao do 3 para o 2 ltm = -0.37
Total de Ebits 20
Total de Trimestres 20


In [7]:
ebitda = []
for i in range(len(trimestres_lista)):
    ebit_value = transform_Dfp_tri(doc_stock, '3.05', i, len(trimestres_lista), trimestres_lista, tipo_balanco1, tipo_balanco2) 
    depreciation_value = depreciacao_FDC_tri(doc_stock, i, len(trimestres_lista), trimestres_lista, tipo_balanco1, tipo_balanco2) 
    ebitda.append(ebit_value+depreciation_value)
ebitda = np.array(ebitda)

print(stock)

# calcular 3t, 2t, 1t variacao
info_input.append(round(ebitda[0]/ebitda[0+4], 2)) # 11
info_input.append(round(ebitda[1]/ebitda[1+4], 2)) # 12 
info_input.append(round(ebitda[2]/ebitda[2+4], 2)) # 13
info_input.append(round(info_input[12] - info_input[13], 2)) # 14
info_input.append(round(info_input[11] - info_input[12], 2)) # 15
print(f'Trimestre {trimestres_lista[0]} com variação de {info_input[11]}')
print(f'Trimestre {trimestres_lista[1]} com variação de {info_input[12]}')
print(f'Trimestre {trimestres_lista[2]} com variação de {info_input[13]}')
print(f'Variacao do 2 para o 1 = {info_input[14]}')
print(f'Variacao do 3 para o 2 = {info_input[15]}')

# calcular o ebit ltm
info_input.append(round(np.sum(ebitda[0:4])/np.sum(ebitda[4:4+4]), 2)) # 16
info_input.append(round(np.sum(ebitda[1:1+4])/np.sum(ebitda[5:5+4]), 2)) # 17
info_input.append(round(np.sum(ebitda[2:2+4])/np.sum(ebitda[6:6+4]), 2)) # 18
info_input.append(round(info_input[17] - info_input[18], 2)) # 19
info_input.append(round(info_input[16] - info_input[17], 2)) # 20

print(f'Trimestre {trimestres_lista[0]} com variação LTM de {info_input[16]}')
print(f'Trimestre {trimestres_lista[1]} com variação LTM de {info_input[17]}')
print(f'Trimestre {trimestres_lista[2]} com variação LTM de {info_input[18]}')
print(f'Variacao do 2 para o 1 ltm = {info_input[19]}')
print(f'Variacao do 3 para o 2 ltm = {info_input[20]}')

print(f'Total de Ebitda {len(ebitda)}')
print(f'Total de Trimestres {len(trimestres_lista)}')

VAMO3
Trimestre 2023-09-30 com variação de 1.23
Trimestre 2023-06-30 com variação de 1.48
Trimestre 2023-03-31 com variação de 1.82
Variacao do 2 para o 1 = -0.34
Variacao do 3 para o 2 = -0.25
Trimestre 2023-09-30 com variação LTM de 1.54
Trimestre 2023-06-30 com variação LTM de 1.74
Trimestre 2023-03-31 com variação LTM de 1.85
Variacao do 2 para o 1 ltm = -0.11
Variacao do 3 para o 2 ltm = -0.2
Total de Ebitda 20
Total de Trimestres 20


In [8]:
fluxocaixa_operacional = []
for i in range(len(trimestres_lista)):
    fluxocaixa_value = transform_FDC_tri(doc_stock, '6.01.01', i, len(trimestres_lista), trimestres_lista, tipo_balanco1, tipo_balanco2)
    fluxocaixa_operacional.append(fluxocaixa_value)
fluxocaixa_operacional = np.array(fluxocaixa_operacional)

print(stock)

# calcular 3t, 2t, 1t variacao
info_input.append(round(fluxocaixa_operacional[0]/fluxocaixa_operacional[0+4], 2)) # 21
info_input.append(round(fluxocaixa_operacional[1]/fluxocaixa_operacional[1+4], 2)) # 22
info_input.append(round(fluxocaixa_operacional[2]/fluxocaixa_operacional[2+4], 2)) # 23
info_input.append(round(info_input[22] - info_input[23], 2)) # 24
info_input.append(round(info_input[21] - info_input[22], 2)) # 25
print(f'Trimestre {trimestres_lista[0]} com variação de {info_input[21]}')
print(f'Trimestre {trimestres_lista[1]} com variação de {info_input[22]}')
print(f'Trimestre {trimestres_lista[2]} com variação de {info_input[23]}')
print(f'Variacao do 2 para o 1 = {info_input[24]}')
print(f'Variacao do 3 para o 2 = {info_input[25]}')

# calcular o ebit ltm
info_input.append(round(np.sum(fluxocaixa_operacional[0:4])/np.sum(fluxocaixa_operacional[4:4+4]), 2)) # 26
info_input.append(round(np.sum(fluxocaixa_operacional[1:1+4])/np.sum(fluxocaixa_operacional[5:5+4]), 2)) # 27
info_input.append(round(np.sum(fluxocaixa_operacional[2:2+4])/np.sum(fluxocaixa_operacional[6:6+4]), 2)) # 28
info_input.append(round(info_input[27] - info_input[28], 2)) # 29
info_input.append(round(info_input[26] - info_input[27], 2)) # 30

print(f'Trimestre {trimestres_lista[0]} com variação LTM de {info_input[26]}')
print(f'Trimestre {trimestres_lista[1]} com variação LTM de {info_input[27]}')
print(f'Trimestre {trimestres_lista[2]} com variação LTM de {info_input[28]}')
print(f'Variacao do 2 para o 1 ltm = {info_input[29]}')
print(f'Variacao do 3 para o 2 ltm = {info_input[30]}')

print(f'Total de Fluxo de Caixa Operacional {len(fluxocaixa_operacional)}')
print(f'Total de Trimestres {len(trimestres_lista)}')

VAMO3
Trimestre 2023-09-30 com variação de 1.32
Trimestre 2023-06-30 com variação de 1.47
Trimestre 2023-03-31 com variação de 1.97
Variacao do 2 para o 1 = -0.5
Variacao do 3 para o 2 = -0.15
Trimestre 2023-09-30 com variação LTM de 1.61
Trimestre 2023-06-30 com variação LTM de 1.84
Trimestre 2023-03-31 com variação LTM de 2.02
Variacao do 2 para o 1 ltm = -0.18
Variacao do 3 para o 2 ltm = -0.23
Total de Fluxo de Caixa Operacional 20
Total de Trimestres 20


In [9]:
# Encontrar data de divulgacao dos nrs ou definir datas fixas // optar por datas fixas ou variáveis
# document_inforesultados = finddoc_byname(db, 'divulgacaoinfo_resultados', id_empresa, '_30/06/2023').next()
document_inforesultados = finddoc_by_codcvm(db, 'divulgacaoinfo_resultados', doc_stock['cod_cvm'], id_empresa, '_30/06/2023')
print(type(document_inforesultados))

nome_cia = document_inforesultados['nome_empresa']
hora_envioresultado = document_inforesultados['hora_envio']

info_input.append(converterstr_data(hora_envioresultado) + timedelta(extrair_hora(hora_envioresultado))) # '2023-08-10' # 31
info_input.append('2023-10-10') # 32
info_input.append(get_adjclosing_price(df_stocks, str(info_input[31]), stock)) # 33
info_input.append(get_adjclosing_price(df_stocks, info_input[32], stock)) # 34
info_input.append(round(info_input[34]/ info_input[33]- 1, 4)) # 35

print(f'Data Inicial: {info_input[31]} e Data Final: {info_input[32]}')
print(f'Preço Inicial: {info_input[33]} e Preço Final: {info_input[34]}')
print(f'Retorno {stock} : {info_input[35]}')

## IBOVESPA
preco_inicial = get_adjclosing_price(df_stocks, str(info_input[31]), 'BOVA11')
preco_final = get_adjclosing_price(df_stocks, info_input[32], 'BOVA11')
info_input.append(round(preco_final/ preco_inicial- 1, 4)) # 36
print(f'Retorno BOVA11 : {info_input[36]}')

## SMALL
preco_inicial = get_adjclosing_price(df_stocks, str(info_input[31]), 'SMAL11')
preco_final = get_adjclosing_price(df_stocks, info_input[32], 'SMAL11')
info_input.append(round(preco_final/ preco_inicial- 1, 4)) # 37
print(f'Retorno SMAL11 : {info_input[37]}')

re.compile('VAMOS LOCAÇÃO DE CAMINHÕES, MÁQUINAS E EQUIPAMENTOS S.A._30/06/2023', re.IGNORECASE)
CODIGO EXISTENTE EM MAIS DE 01 DOCUMENTO
<class 'dict'>
Data Inicial: 2023-08-04 e Data Final: 2023-10-10
Preço Inicial: 12.12 e Preço Final: 9.16
Retorno VAMO3 : -0.2442
Retorno BOVA11 : -0.0232
Retorno SMAL11 : -0.1185


In [10]:
def get_adjclosing_price(df, date, ticker):
    if isinstance(date, str):
        date = pd.to_datetime(date)
    filtered_df = df[(df['data'] == date) & (df['ticker'] == ticker)]
    if not filtered_df.empty:
        return filtered_df['preco_fechamento_ajustado'].iloc[0]
    else:
        return None
    
filtered_df = df_stocks[(df_stocks['ticker'] == stock)]
print(str(info_input[31]))
filtered_df.tail(5)


2023-08-04


,data,preco_abertura,preco_fechamento,preco_maximo,preco_medio,preco_minimo,quantidade_negociada,quantidade_negocios,ticker,volume_negociado,fator_ajuste,preco_fechamento_ajustado
1604922,2023-09-06,11.32,10.73,11.33,10.95,10.65,30094100.0,42359.0,VAMO3,329619759.0,1.0,10.73
1604923,2023-09-08,10.80,10.65,10.92,10.67,10.52,10263500.0,25149.0,VAMO3,109593490.0,1.0,10.65
1604924,2023-09-11,10.75,10.75,10.87,10.69,10.55,8744700.0,19082.0,VAMO3,93539217.0,1.0,10.75
1604925,2023-09-12,10.81,11.13,11.17,11.03,10.69,7074900.0,17450.0,VAMO3,78037537.0,1.0,11.13
1604926,2023-09-13,11.18,11.07,11.38,11.18,11.07,8336300.0,30895.0,VAMO3,93249738.0,1.0,11.07
1604927,2023-09-14,11.13,11.22,11.37,11.25,11.03,5226800.0,14466.0,VAMO3,58817554.0,1.0,11.22
1604928,2023-09-15,11.22,10.60,11.31,10.75,10.60,14894000.0,15971.0,VAMO3,160159558.0,1.0,10.60
1604929,2023-09-18,10.54,10.00,10.54,10.11,9.92,19156300.0,32423.0,VAMO3,193821254.0,1.0,10.00
1604930,2023-09-19,10.00,9.95,10.12,9.96,9.80,8872800.0,21636.0,VAMO3,88418551.0,1.0,9.95
1604931,2023-09-20,9.97,10.25,10.43,10.27,9.97,8067500.0,22644.0,VAMO3,82923735.0,1.0,10.25


In [11]:
import openpyxl

from openpyxl.styles import Font
from openpyxl.styles import PatternFill
from openpyxl import load_workbook

arquivo = openpyxl.load_workbook('estquant_fundamentus.xlsx')
planilha = arquivo['Planilha1']


# como encontrar a ultima linha disponivel da coluna A
for linha in range(1, planilha.max_row + 1):
    if not planilha[f'A{linha}'].value:
        linha  # Retorna o número da linha da primeira célula em branco


# loop colocando todos os dados na planilha
# variáveis = stock, 
# varebit_3t
# varebit_2t
# varebit_1t

# for i in range(4)

# for j in range(linha+1, 7):
for w in range(1, len(info_input)+1):
    planilha.cell(linha+1, w).value = info_input[w-1]


# planilha[f'A{linha+1}'] = 'RADL3'
arquivo.save('estquant_fundamentus_test.xlsx')
arquivo.close()


In [12]:
def finddoc_by_codcvm(fDb, fCollection_name, fCodcvm, fContaining_word, fTrimestre): 
    count = 0
    while count != 1:
        collection = fDb[fCollection_name]
        documents = collection.find({'cod_cvm': fCodcvm})
        count = collection.count_documents({'cod_cvm': fCodcvm})
        if count > 1:
            regex = re.compile(fContaining_word+fTrimestre, re.IGNORECASE)
            print(regex)
            for doc in documents:
                if fTrimestre in doc['_id']:
                    documents = doc
                    count = 1
                    break
            print('CODIGO EXISTENTE EM MAIS DE 01 DOCUMENTO')
        if count == 0:
            print("Não foi encontrada a empresa desejada.")
            return None      
    if isinstance(documents, Cursor):
        return documents.next()
    else: 
        return documents

def finddoc_byname(fDb, fCollection_name, fContaining_word, fTrimestre=False):
    count = 0
    while count != 1:
        if fTrimestre:
            collection = fDb[fCollection_name]
            regex = re.compile(fContaining_word+fTrimestre, re.IGNORECASE)
            documents = collection.find({'_id': regex})
            count = collection.count_documents({'_id': regex})
        else:
            collection = fDb[fCollection_name]
            regex = re.compile(fContaining_word, re.IGNORECASE)
            documents = collection.find({'_id': regex})
            count = collection.count_documents({'_id': regex})
        if count > 1:
            print(type(documents))
            print('erro aconteceu - checar base')
            for doc in documents:
                print(doc['_id'])
            break
        if count == 0:
            print("Não foi encontrada a empresa desejada.")
            return None
    return documents

# document_inforesultados = finddoc_by_codcvm(db, 'divulgacaoinfo_resultados', doc_stock['cod_cvm'], "GERDAU S.A.", '_30/06/2023').next()
document_inforesultados = finddoc_by_codcvm(db, 'divulgacaoinfo_resultados', doc_stock['cod_cvm'], "GERDAU S.A.", '_30/06/2023')
document_inforesultados

re.compile('GERDAU S.A._30/06/2023', re.IGNORECASE)
CODIGO EXISTENTE EM MAIS DE 01 DOCUMENTO


{'_id': 'VAMOS LOCAÇÃO DE CAMINHÕES, MÁQUINAS E EQUIPAMENTOS S.A._30/06/2023',
 'nome_empresa': 'VAMOS LOCAÇÃO DE CAMINHÕES, MÁQUINAS E EQUIPAMENTOS S.A.',
 'cod_cvm': 24716,
 'nr_sequencial': 129729,
 'data_doc': '30/06/2023',
 'data_envio': '03/08/2023',
 'hora_envio': '03/08/2023 23:45:40',
 'link_consulta': 'https://www.rad.cvm.gov.br/ENET/frmGerenciaPaginaFRE.aspx?NumeroSequencialDocumento=129729&CodigoTipoInstituicao=1',
 'link_download': 'http://www.rad.cvm.gov.br/ENETCONSULTA/frmDownloadDocumento.aspx?CodigoInstituicao=1&NumeroSequencialDocumento=129729'}

In [13]:
# resolver 
# BBDC4
# BBAS3
# ITUB4
# AESB3
# BPAN4 - banco
# BRSR6 - banco
# AGRO3
# CAML3
# BLAU3
# BHIA3
# CBAV3
# SOMA3
# IFCM3
# IRBR3
# CASH3
# JALL3
# MILS3
# MBLY3 
# MEGA3
# SMTO3
# VIVA3
21/138

0.15217391304347827